# Experimentación

En este notebook corremos los experimentos descritos en el informe

In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import subprocess

def run(algorithm: str, instance_path: str) -> (int, float):
    """Corre el programa para la instancia dada y devuelve (resultado, tiempo de ejecucion)."""
    result = subprocess.run(
        f"../build/npm {algorithm} < {instance_path}",
        shell=True, capture_output=True, text=True, check=True,
    )

    return int(result.stdout), float(result.stderr)

In [3]:
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def run_solapamiento(instance_path: str):
    """Corre el programa para la instancia dada y devuelve (resultado, matriz de solapamiento)."""
    result = subprocess.run(
        f"../build/npm GR < {instance_path}",
        shell=True, capture_output=True, text=True, check=True,
    )

    return json.loads(result.stderr.split('\n')[0])["pd_accesses"]


# TODO: arreglar sta mierda
#df1 = pd.DataFrame(run_solapamiento("instancias/grupos/1-grupos_1.in"))
#df25 = pd.DataFrame(run_solapamiento("instancias/grupos/25-grupos_1.in"))

#fig, ax = plt.subplots(ncols=2, sharey=True)
#g = sns.heatmap(data=df25, cbar=True, ax=ax[0])
#g.set_title("25")
#sns.heatmap(data=df1, cbar=True, ax=ax[1])

In [4]:
from typing import List

def get_instances(dataset: str) -> List[str]:
    """Para cada dataset devuelve la lista de instancias correspondientes con el path completo"""
    instances = os.listdir(f"instancias/{dataset}")
    return list(filter(
        lambda i: i.endswith(".in"),
        map(lambda i: f"instancias/{dataset}/"+i, instances),
    ))


In [5]:
def run_instance(rows: list, dataset: str, algorithms: List[str]):
    """
    Corre una instancia para todos los algoritmos, llenando por referencia
    la lista de rows. Lanza una excepcion si para alguna instancia los
    resultados de todos los algoritmos no coinciden.
    """
    for instance in tqdm(get_instances(dataset), "instancias"):
        results = {}
        for alg in algorithms:
            res, t = run(alg, instance)
            rows.append({
                "dataset": dataset,
                "algorithm": alg,
                "time_ms": t,
                "instance": instance,
            })

            results[alg] = res

        if not np.alltrue([results[algorithms[0]] == res for res in results.values()]):
            print(f"Ojo que con la instancia {instance} no dieron todos iguales. Resultados: {results}")

## Implementación de la experimentación

In [6]:
ALL = ["FB", "BT", "BT-F", "BT-O-G", "BT-O-C", "DP"]

In [7]:
# type experimento struct {
#     algoritmos: []string
#     dataset: string
# }

experimentos = [
    {
        "algoritmos" : ALL,
        "dataset"    : "control",
    },
    {
    # DP para ver cómo afecta el solapamiento y BT debería ser siempre más o menos igual
    
    # TODO: revisar la matriz de solapamiento
        "algoritmos" : ["BT", "DP"],
        "dataset"    : "grupos",
    },
    # Optimalidad
    {
        "algoritmos" : ALL,
        "dataset"    : "one-to-rule",
    },
    {
        "algoritmos" : ALL,
        "dataset"    : "identicos",
    },
    # Factibilidad
    {
        "algoritmos" : ALL,
        "dataset"    : "low-M",
    },
    # Programación Dinámica
    {
        "algoritmos" : ["BT-F", "DP"],
        "dataset"    : "cache",
    },
    {
        "algoritmos" : ["DP"],
        "dataset"    : "complejidad-DP",
    }
]


In [8]:
import os
import numpy as np
import pandas as pd

from tqdm import tqdm

rows = []

for i, experimento in enumerate(experimentos):
    print("Corriendo el experimento {0} ({1}/{2})".format(experimento["dataset"], i+1, len(experimentos)))
    run_instance(rows, experimento["dataset"], experimento["algoritmos"])

df_results = pd.DataFrame(rows, columns=["dataset", "algorithm", "time_ms", "instance"])
print("Finished!")

instancias:   3%|▎         | 2/78 [00:00<00:04, 18.37it/s]

Corriendo el experimento control (1/7)


instancias:   1%|          | 7/1365 [00:00<00:24, 56.17it/s]

Corriendo el experimento grupos (2/7)


instancias:   0%|          | 0/90 [00:00<?, ?it/s]

...

Corriendo el experimento cache (6/7)


instancias:   1%|          | 10/1200 [00:00<00:13, 90.90it/s]

Corriendo el experimento complejidad-DP (7/7)


instancias: 100%|██████████| 1200/1200 [00:15<00:00, 79.46it/s]

Finished!


In [9]:
df_results.to_csv("resultados.csv")